In [1]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import geopandas as gpd

# get data: controls and outcomes

In [2]:
controls = pd.read_csv('../data/glavni/controls.csv')
# replace quotes in column names
controls.columns = controls.columns.str.replace('"', '')
controls.columns = controls.columns.str.replace(':', '')
controls.columns = controls.columns.str.replace(',', '')
controls.rename(columns={c:"c_"+c for c in controls.columns if not c in ['geography code', 'geography']}, inplace=True)

controls = controls.drop_duplicates()


len(controls), len(controls.drop_duplicates()), len(set(controls.index)), len(controls['geography code'].unique())

(35672, 35672, 35672, 35672)

In [3]:
controls.columns

Index(['geography code', 'c_percent asian', 'c_percent black',
       'c_percent mixed', 'c_percent white', 'c_percent christian',
       'c_percent jewish', 'c_percent no religion', 'c_percent muslim',
       'c_percent no central heating', 'c_percent wood heating',
       'c_percent communal heating', 'c_percent TFW less than 2km',
       'c_percent TFW 2km to 5km', 'c_percent TFW 60km and over',
       'c_percent WFH', 'c_percent part-time',
       'c_percent 15 hours or less worked',
       'c_percent 49 or more hours worked', 'c_percent commute on foot',
       'c_percent commute bus', 'c_percent commute bicycle',
       'c_percent same address', 'c_percent student moved to address',
       'c_percent occupancy rating bedrooms +2',
       'c_percent occupancy rating bedrooms 0',
       'c_percent occupancy rating bedrooms -2',
       'c_percent occupancy rating rooms +2',
       'c_percent occupancy rating rooms 0',
       'c_percent occupancy rating rooms -2',
       'c_percent 1

In [12]:
outcomes = pd.read_csv('../data/glavni/outcomes.csv')
outcomes.rename(columns={c:"o_"+c for c in outcomes.columns if not c in ['geography code', 'geography']}, inplace=True)

outcomes = outcomes.drop_duplicates()

len(outcomes), len(outcomes.drop_duplicates()), len(set(outcomes.index)), len(outcomes['geography code'].unique())

(32833, 32833, 32833, 32833)

In [13]:
env = pd.read_csv('../data/glavni/environment.csv').rename(columns={'LSOA21CD':'geography code'})
del env['LSOA21NM']
env.rename(columns={c:"e_"+c for c in env.columns if not c in ['geography code', 'geography']}, inplace=True)

del env['e_Snow and ice']
del env['e_Mangroves']

env.rename(columns={'e_Bare / sparse vegetation':'e_Bare sparse vegetation'}, inplace=True)
# del env['e_evaporation_from_open_water_surfaces_excluding_oceans_sum']

env = env.drop_duplicates()

len(env), len(env.drop_duplicates()), len(set(env.index)), len(env['geography code'].unique())

(36778, 36778, 36778, 33804)

In [14]:
env.columns

Index(['geography code', 'e_NO2', 'e_ozone',
       'e_total_aerosol_optical_depth_at_550nm_surface',
       'e_particulate_matter_d_less_than_25_um_surface', 'e_ndvi',
       'e_dewpoint_temperature_2m', 'e_temperature_2m',
       'e_soil_temperature_level_1', 'e_soil_temperature_level_3',
       'e_lake_bottom_temperature', 'e_lake_mix_layer_depth',
       'e_lake_mix_layer_temperature', 'e_lake_total_layer_temperature',
       'e_snow_albedo', 'e_snow_cover', 'e_snow_density', 'e_snow_depth',
       'e_skin_reservoir_content', 'e_volumetric_soil_water_layer_1',
       'e_volumetric_soil_water_layer_3', 'e_surface_latent_heat_flux_sum',
       'e_surface_net_solar_radiation_sum',
       'e_surface_solar_radiation_downwards_sum',
       'e_surface_thermal_radiation_downwards_sum',
       'e_evaporation_from_bare_soil_sum',
       'e_evaporation_from_the_top_of_canopy_sum',
       'e_evaporation_from_open_water_surfaces_excluding_oceans_sum',
       'e_total_evaporation_sum', 'e_u_comp

In [15]:
env=env.fillna(0.0)

# merge

In [16]:
data = controls.merge(outcomes, on=['geography code'])
data = data.merge(env, on=['geography code'])

In [17]:
len(data), len(data.drop_duplicates()), len(set(data.index)), len(data['geography code'].unique())

(34670, 34670, 34670, 31799)

# remove duplicates

In [18]:
# group the dataframe by 'geography code' and select the first element from each group
# data = data.reset_index().rename(columns={'index': 'geography code'})
data = data.groupby('geography code').first()
data = data.reset_index().rename(columns={'index': 'geography code'})
len(data), len(data.drop_duplicates()), len(set(data.index)), len(data['geography code'].unique())

(31799, 31799, 31799, 31799)

## save raw master

In [19]:
data.to_csv('../data/glavni/raw_master.csv', index=None)

# create a test / train split and save for future uses

In [36]:
df = data.set_index('geography code')

# Separate features and target variable
X = df.filter(regex='^(c_|e_)') # Select columns starting with 'c_' or 'e_'
y = df.filter(regex='^(o_)') # Select columns starting with 'c_' or 'e_'

# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=44)

# Save train and test sets
train = X_train.join(y_train).drop_duplicates()
# train.to_csv('../data/glavni/train_raw_master.csv')
test = X_test.join(y_test).drop_duplicates()
# test.to_csv('../data/glavni/test_raw_master.csv')

len(set(X_train.index)), len(set(y_train.index)), len(X_test.index), len(y_test.index)


# save
train.to_csv('../data/glavni/train_raw_master.csv')
test.to_csv('../data/glavni/test_raw_master.csv')

In [22]:
len(data), len(train), len(test)

(31799, 25439, 6360)

# SPATIAL MASTER

In [23]:
# tmp = data[data['geography code']=='E01029797'].drop_duplicates()

In [24]:
# env_col = [c for c in tmp.columns if c.startswith('e_')]
# tmp[env_col].hist(figsize=(20,20));

In [25]:
# duplicated_rows = tmp[tmp.duplicated()]
# print(duplicated_rows)

# Add Region for train test split spatially

In [26]:
regions = gpd.read_file('../data/GEO_DATA/2021/Regions_(December_2022)_EN_BFC/Regions_(December_2022)_EN_BFC.shp')
lsoas = gpd.read_file('../data/GEO_DATA/2021/LSOA_(Dec_2021)_Boundaries_Generalised_Clipped_EW_(BGC)/LSOA_(Dec_2021)_Boundaries_Generalised_Clipped_EW_(BGC).shp')

In [27]:
# len(lsoas['LSOA21CD'].unique()), len(lsoas['LSOA21CD'])

# spatial join LSOA REGION

In [28]:
lsoas_regions = gpd.sjoin(lsoas, regions, predicate='within')

In [29]:
len(lsoas_regions['LSOA21CD']), len(set(lsoas_regions['LSOA21CD']))

(30523, 30523)

In [30]:
lsoas_regions_mapping = lsoas_regions[['LSOA21CD', 'RGN22CD', 'RGN22NM', 'LSOA21NM']].drop_duplicates()

## save the mapping

In [31]:
lsoas_regions_mapping.to_csv('../data/glavni/lsoas_regions_mapping.csv', index=None)

## add region to data

In [32]:
spatial_data = data.merge(lsoas_regions_mapping.\
                rename(columns={'LSOA21CD':'geography code', 'RGN22NM':'region'})\
                [['geography code', 'region']],\
                 on='geography code')

In [33]:
spatial_data = spatial_data.set_index('geography code')
spatial_data.head()

,c_percent asian,c_percent black,c_percent mixed,c_percent white,c_percent christian,c_percent jewish,c_percent no religion,c_percent muslim,c_percent no central heating,c_percent wood heating,...,e_Tree cover,e_Shrubland,e_Grassland,e_Cropland,e_Built-up,e_Bare sparse vegetation,e_Permanent water bodies,e_Herbaceous wetland,e_Moss and lichen,region
geography code,,,,,,,,,,,,,,,,,,,,,
E01000001,0.107191,0.007463,0.037992,0.801221,0.345763,0.019661,0.483390,0.021695,0.026190,0.000000,...,15.994640,0.0,0.000000,0.000000,82.937771,0.000000,1.067589,0.0,0.0,London
E01000002,0.130592,0.007937,0.043290,0.782107,0.338150,0.025289,0.491329,0.018786,0.019417,0.001214,...,3.367407,0.0,0.027592,0.000000,96.133342,0.055184,0.416476,0.0,0.0,London
E01000003,0.094913,0.034739,0.062655,0.741315,0.340360,0.026658,0.479851,0.030998,0.035329,0.000000,...,14.992813,0.0,0.108761,0.000000,84.898426,0.000000,0.000000,0.0,0.0,London
E01000005,0.321526,0.108084,0.071753,0.385104,0.367514,0.012704,0.221416,0.309437,0.012526,0.000000,...,1.286766,0.0,0.065390,0.000000,98.647845,0.000000,0.000000,0.0,0.0,London
E01000006,0.479675,0.108401,0.037940,0.327913,0.371614,0.000542,0.070423,0.391116,0.027076,0.000000,...,11.504611,0.0,3.877869,0.215842,84.401679,0.000000,0.000000,0.0,0.0,London


In [34]:
len(spatial_data)

28777

## save spatial master

In [35]:
spatial_data.to_csv('../data/glavni/spatial_raw_master.csv')

# Inspect Raw Data

In [ ]:
data.hist(figsize=(35,60), layout=(24,5));

In [ ]:
raw_data = pd.read_csv('../data/glavni/raw_master.csv')
data= raw_data.copy()

# Log Data

In [ ]:
columns_to_log_part1 = ['c_percent asian', 'c_percent black',\
                 'c_percent jewish','c_percent muslim', 'c_percent no central heating',\
                 'c_percent communal heating', 'c_percent TFW 60km and over',\
                 'c_percent commute bus', 'c_percent commute bicycle',\
                 'c_percent poor-english', \
                 'c_percent highly-deprived',]

columns_to_log_part2 = ['o_OME_per_capita', 'c_pop_density',\
                     'o_total_quantity_per_capita','c_total population']

columns_to_log_part3 = ['percent same address', 'percent student moved to address',\
                        'percent occupancy rating bedrooms: 0', 'percent occupancy rating bedrooms: -2',\
                        'percent occupancy rating rooms: 0', 'percent occupancy rating rooms: -2',\
                        'percent 1. Managers, directors and senior officials', 'percent 2. Professional occupations',\
                        'percent 6. Caring, leisure and other service occupations', 'percent 7. Sales and customer service occupations',\
                        'percent 9. Elementary occupations', \
                        'percent born in the UK', 'percent 10 years or more',\
                        'percent 2 years or more, but less than 5 years',\
                        'percent less than 2 years']
columns_to_log_part3 = ['c_'+c for c in columns_to_log_part3]

columns_to_log_part4 = ['e_snow_cover', 'e_snow_depth', 'e_surface_net_solar_radiation_sum',\
                        'e_surface_solar_radiation_downwards_sum', 'e_evaporation_from_the_top_of_canopy_sum',\
                        'e_evaporation_from_open_water_surfaces_excluding_oceans_sum',\
                        'e_total_precipitation_sum', 'e_surface_runoff_sum', \
                        'e_Tree cover', 'e_Shrubland', 'e_Grassland', 'e_Cropland',\
                        'e_Built-up', 'e_Bare sparse vegetation', \
                        'e_Permanent water bodies', 'e_Herbaceous wetland', \
                        'e_Moss and lichen', 'e_surface_pressure']

In [ ]:
data_logged = data.copy()
for c in columns_to_log_part1:
    data_logged[c] = data[c].apply(lambda x: np.log(x+0.01))

In [ ]:
# data_logged = data.copy()
for c in columns_to_log_part2:
    data_logged[c] = data[c].apply(lambda x: np.log(x))

In [ ]:
# data_logged = data.copy()
for c in columns_to_log_part3:
    data_logged[c] = data[c].apply(lambda x: np.log(x+0.01))

In [ ]:
# data_logged = data.copy()
for c in columns_to_log_part4:
    data_logged[c] = data[c].apply(lambda x: np.log(x+1))

In [ ]:
data_logged.hist(figsize=(35,60), layout=(24,5));

## Save Logged Master

In [ ]:
data_logged.to_csv('../data/glavni/log_master.csv', index=None)

# Standardize

In [ ]:
data_standardized = data_logged.copy()

features_ztransform = [c for c in data_standardized.columns if not c in ['geography code', 'geography']]

for k in features_ztransform:
    data_standardized[k] = (data_standardized[k] -\
            data_standardized[k].mean())/data_standardized[k].std(ddof=0)

In [ ]:
data_standardized.hist(figsize=(35,60), layout=(24,5));

# Save

In [ ]:
data_standardized.to_csv('../data/glavni/standardized_master.csv', index=None)

# MinMax Normalize

In [ ]:
data_normalized = data_logged.copy().dropna()

features_transform = [c for c in data_normalized.columns if not c in ['geography code', 'geography']]

scaler = MinMaxScaler()

data_normalized[features_transform] = scaler.fit_transform(data_normalized[features_transform])

In [ ]:
data_normalized.describe()

In [ ]:
data_normalized.hist(figsize=(35,60), layout=(24,5));

In [ ]:
data_normalized.to_csv('../data/glavni/normalized_master.csv', index=None)